## 计算图
- 将代码分解成操作子
- 将计算表示成一个无环图
- 显示构造
- 隐示构造

## 自动求导的两种模式
- 链式法则
- 正向累积 （先算$\frac{\partial{u_2}}{\partial{u_1}}$）
- 反向累积（反向传递）(先算$\frac{\partial{y}}{\partial{u_n}}$)

## 复杂度
计算复杂度和内存复杂度O(n)

In [2]:
import torch

x = torch.arange(4.0)
x

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True)
x.grad #默认None

In [4]:
y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [5]:
# 调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward()
x.grad # 计算得到的梯度

tensor([ 0.,  4.,  8., 12.])

In [6]:
x.grad == 4 * x # 计算得到的梯度与手动计算的梯度相同

tensor([True, True, True, True])

In [7]:
# 在默认情况下，pytorch会累积梯度，需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad  

tensor([1., 1., 1., 1.])

In [8]:
x.grad.zero_()
y = x * x # 哈达玛积，实际上y是一个向量
y.sum().backward() # 对y求和先得到标量
x.grad # 计算得到的梯度

tensor([0., 2., 4., 6.])

In [9]:
# 将某些计算移动到记录的计算图之外
x.grad.zero_()
y = x * x
u = y.detach() # 断开与计算图的连接,**u不再是x的函数，而是一个常数**
z = u * x

z.sum().backward() 
x.grad == u # z对x的梯度等于u

tensor([True, True, True, True])

In [10]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x # y对x的梯度等于2x

tensor([True, True, True, True])

In [ ]:
# python控制流的自动微分
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)   # size为空代表标量，a是一个随机数
d = f(a)
d.backward()

a.grad == d / a 

tensor(True)